In [3]:
#test jupyternotebook- now all functions EXCEPT classify_das.py can be run as individual functions
from DAS_extract_V2 import grab_PMIDs_from_csv
from DAS_extract_V2 import isolate_rows_by_PMID
from DAS_extract_V2 import extract_files
from DAS_extract_V2 import get_nxml
from DAS_extract_V2 import citation_count
import pandas as pd
import numpy as np

#a test to see if it works in jupyter notebook
file = "example_files/PMC_parser_output.csv"
df1 = pd.read_csv(file)
df2 = df1['DOI']
df1['citation_count'] = citation_count(df2)
print(df1)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/Users/muthuku/Desktop/final_xml_07_22'

In [ ]:
# %run /path/to/my_script.py FOR CLASSIFY DAS python file 
# !python /path/to/my_script.py

In [4]:
#STEP 1 -grab PMIDs from CSHL digital repo CSV 
from DAS_extract_V2 import grab_PMIDs_from_csv
from DAS_extract_V2 import isolate_rows_by_PMID
from DAS_extract_V2 import extract_files
from DAS_extract_V2 import get_nxml

datafile = "example_files/2023_07_06_CSHL_articles_2007-2022_from_IR.csv"
pmids_output_file = "example_files/pmids_07_22.txt"

grab_PMIDs_from_csv(datafile,pmids_output_file)

#STEP2 check OA database CSV for files to download-oa_file_list was not able to be uploaded to Github, you must download your own copy from
#go to the Base FTP URL https://ftp.ncbi.nlm.nih.gov/pub/pmc/
isolate_rows_by_PMID('example_files/pmids_07_22.txt', 'example_files/oa_file_list.csv', 'example_files/output1_07_22_PMC.csv')

#STEP 3 - using the links in the output file and download zipped PMC folders containing .nxml, .pdf, figures and data into a directory (download_PMC fxn)
#STEP 3A- Each downladed file will look like PMC{ID Number}.tar.gz, using extract_files function to unzip them 

download_PMC('example_files/output1_07_22_PMC.csv', "PMC_folder_07_22")
folder = 'example_files/PMC_folder_07_22'
extract_files(folder)

#STEP 4 grab the full text XML from each PMC folder and create a folder with all articles
source_folder = 'example_files/PMC_folder_07_22'
target_folder = 'example_files/final_xml_07_22'
file_extension = '.nxml'

get_nxml(source_folder,target_folder,file_extension)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/Users/muthuku/Desktop/final_xml_07_22'

In [ ]:
from DAS_extract_V2 import parse_xml_directory
#IN THIS STEP WE ARE PARSING THROUGH EACH FULL TEXT XML and returning key article metadata including DAS and url links

xml_directory = '/Users/muthuku/Desktop/final_xml_07_22'
df1 = parse_xml_directory(xml_directory)
#DATAFRAME cleanup steps - run these after the main function above
# create a combined DAS column to include results from all methods of DAS collection 
df1['combined_DAS'] = df1['Data Availability 1'] + df1['Data Availability 2'] + df1['Data Availability 3'] + df1['Data Availability 4'] + df1['Data Availability 5'] 
df1['unique_combined_das'] = df1['combined_DAS'].apply(lambda x: list(set(x)))
count = 0
for n in df1['unique_combined_das']:
	count += 1
	if n == []:
		n.append("Not applicable " + str(count))

#combine URLs as well 
df1['all_urls'] = df1['URL1'] +df1['URL2'] + df1['URL3'] +df1['URL4'] + df1['URL5']
#drop individual columns 
column_list = ['Data Availability 1','Data Availability 2','Data Availability 3','Data Availability 4','Data Availability 5','URL1', 'URL2','URL3','URL4','URL5', 'combined_DAS']
df2= df1.drop(columns = column_list)
#convert the list of seperate strings into a single string- join all sentences together and remove commas which interfere with ML 
df2['combined_string_DAS'] = df2['unique_combined_das'].apply(lambda x: ' '.join(x))
df2['combined_string_DAS'] = df2['combined_string_DAS'].str.replace(',', '')
df2 = df2.drop(['unique_combined_das'], axis = 1)
df2.to_csv("PMC_parser_output.csv", encoding = "utf-8")

In [ ]:
from DAS_extract_V2 import citation_count
#STEP3 -citation counts for each article using SCOPUS API and using article DOIs as an input 
file = "example_files/PMC_parser_output.csv"
df1 = pd.read_csv(file)
df2 = df1['DOI']
df1['citation_count'] = citation_count(df2)
print(df1)

df1.to_csv('og_df_with_citations1.csv', encoding = "utf-8", index = False

In [ ]:
%run ./DAS_extract_V2/classify_das2.py 
#!python3 ./DAS_extract_V2/classify_das2.py

